<a href="https://www.kaggle.com/code/bilgeesinakbaba/fishclassification?scriptVersionId=203039269" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1. Kütüphanelerin Yüklenmesi
Bu adımda, projemizde kullanacağımız çeşitli Python kütüphanelerini yüklüyoruz. Bu kütüphaneler, veriyi işlemek, görselleştirmek ve yapay sinir ağı (ANN) modelini oluşturmak için kullanılacak. Örneğin, numpy ve pandas veri manipülasyonu için, seaborn ve matplotlib ise görselleştirme amacıyla kullanılacak. tensorflow kütüphanesi ise yapay sinir ağı modelini inşa etmek ve eğitmek için kullanılacak.

In [ ]:
# Gerekli kütüphaneleri yükleme
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Input

# 2. Veri Seti Yollarının Tanımlanması
Bu aşamada, veri setinin bulunduğu dosya yolunu ve sınıflarımızı (yani kategorilerimizi) tanımlıyoruz. Her bir kategori, bir balık türünü temsil ediyor ve bu kategoriler daha sonra modelin çıktısı olarak kullanılacak. Kategoriler, modelin sınıflandırmak için öğrenmesi gereken sınıfları temsil eder. Bu işlem, veri yüklemesi sırasında etiketleri belirlememize yardımcı olacak.

In [ ]:
# Veri seti yolu ve kategorilerin tanımlanması
data_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
categories = ['Black Sea Sprat', 'Gilt-Head Bream', 'Hourse Mackerel', 'Red Mullet', 
              'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']

# 3. Veri Yükleme ve Ön İşleme
Bu adımda, her bir kategoriye ait görüntüleri veri setinden yüklüyor ve bu görüntüleri 128x128 boyutuna yeniden boyutlandırıyoruz. Aynı zamanda, bu resimleri vektörleştirerek modelin işleyebileceği hale getiriyoruz. Her kategori için etiketler oluşturularak, modelin sınıflandırma yapması sağlanacak. Ayrıca, veri yükleme sırasında hata oluşabilecek durumları kontrol ediyoruz.

In [ ]:
# Veriyi ve etiketleri depolamak için boş listeler
data = []
labels = []

# Klasörlerden resimleri yükle
for category in categories:
    folder_path = os.path.join(data_dir, category)
    
    # Klasörün var olup olmadığını kontrol et
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        continue

    # Klasör ve alt klasörlerdeki png resim sayısını tutmak için sayaç
    png_count = 0

    # Klasör ve alt klasörlerdeki resimleri yükle
    for root, dirs, files in os.walk(folder_path):
        
        # 'GT' içeren klasörleri atla
        if 'GT' in root:
            continue
            
        for img_name in files:
            img_path = os.path.join(root, img_name)
            if img_name.endswith('.png'):
                try:
                    img = Image.open(img_path).resize((128, 128))  # Resimleri yeniden boyutlandır
                    img = np.array(img)
                    if img.ndim == 3:
                        data.append(img.flatten())  # Resmi düzleştir (1D vektör)
                        labels.append(categories.index(category))  # Kategoriye göre etiketle
                        png_count += 1  # Her png dosyası için sayaç artır
                    else:
                        print(f"Image format not supported or corrupted: {img_path}")
                except Exception as e:
                    print(f"Error: {img_path}, {e}")
    
    # Klasör başına bulunan png sayısını yazdır
    print(f"{category} klasöründe toplam {png_count} PNG resim bulundu.")

# 4. Verilerin Numpy Array Formatına Dönüştürülmesi
Yüklediğimiz veriler ve etiketler liste formatında depolandı. Şimdi bu verileri numpy array formatına dönüştürüyoruz ve bu veriler üzerinde normalizasyon işlemi yapıyoruz. Normalizasyon, görüntü verilerinin 0-255 arasındaki piksel değerlerini 0-1 arasına çekerek modelin daha iyi öğrenmesini sağlar. Aynı zamanda etiketlerimizi one-hot encoding işlemi ile kategorik hale getiriyoruz.

In [ ]:
# Veriyi numpy array'e dönüştür
data = np.array(data)
labels = np.array(labels)

# Veriyi normalleştir (0-255 arası değerleri 0-1 arası yap)
data = data / 255.0

# One-hot encoding işlemi
labels = to_categorical(labels, num_classes=len(categories))

# Veri ve etiketlerin boyutunu kontrol et
print(f"Veri boyutu: {data.shape}")
print(f"Etiket boyutu: {labels.shape}")

# 5. Eğitim ve Test Setlerine Ayırma
Bu adımda, verileri eğitim ve test setlerine ayırıyoruz. Modeli eğitirken verinin %80'i eğitim için, %20'si ise test için kullanılacak. Eğitim seti modelin öğrenmesi için kullanılırken, test seti modelin doğruluğunu değerlendirmek için kullanılacak.

In [ ]:
# Eğer data ve labels boş değilse eğitim ve test setlerine ayırma
if len(data) > 0 and len(labels) > 0:
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
    
    # Eğitim ve test setlerinin boyutlarını kontrol et
    print(f"Eğitim veri boyutu: {X_train.shape}")
    print(f"Test veri boyutu: {X_test.shape}")
else:
    print("Data veya labels listesi boş, lütfen kontrol edin.")

# 6. Modelin Oluşturulması
Bu aşamada, yapay sinir ağı modelini (ANN) oluşturuyoruz. Giriş katmanı, görüntü verilerinin düzleştirilmiş bir formunu alacak, ardından her biri farklı sayıda nörona sahip birkaç gizli katman ekleniyor. Bu gizli katmanlar, modelin verilerden özellikler öğrenmesine yardımcı olacak. Dropout katmanları, aşırı öğrenmeyi (overfitting) önlemek amacıyla kullanılıyor. Çıkış katmanında ise sınıfları temsil eden softmax aktivasyon fonksiyonu kullanıyoruz.

In [ ]:
# Modeli oluşturma
model = Sequential()

# Giriş katmanı: Görüntüleri düzleştiriyoruz
model.add(Input(shape=(128 * 128 * 3,)))  # Giriş şekli düzleştirilmiş görüntüler

# İlk Dense katmanı
model.add(Dense(512, activation='relu')) 
model.add(Dropout(0.1))  # Dropout ile overfitting önlenir

# İkinci Dense katmanı
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.1))  # Dropout ile overfitting önlenir

# Üçüncü Dense katmanı
model.add(Dense(128, activation='relu'))  
model.add(Dropout(0.1))  # Dropout ile overfitting önlenir

# Çıkış katmanı
model.add(Dense(len(categories), activation='softmax'))  

# 7. Modelin Derlenmesi
Bu adımda, oluşturduğumuz yapay sinir ağı modelini derliyoruz. Modeli derlerken optimizasyon ve hata hesaplama yöntemlerini tanımlıyoruz. Adam optimizasyon algoritması, derin öğrenme modellerinde sıkça kullanılan ve öğrenme hızını optimize eden bir yöntemdir. Categorical Crossentropy ise sınıflandırma problemlerinde kullanılan bir kayıp fonksiyonudur. Bu kayıp fonksiyonu, modelin tahminlerinin doğruluğunu değerlendirerek, modelin nasıl güncelleneceğini belirler. Ayrıca, model.summary() fonksiyonu sayesinde, modelin mimarisini özetleyen bir tablo elde ediyoruz. Bu tabloda katmanların özellikleri, parametre sayıları ve çıktılarının şekilleri gösteriliyor. Bu, modelin yapısının doğru olduğundan emin olmamıza yardımcı olur.

In [ ]:
# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli özetle
model.summary()

# 8. Modelin Eğitilmesi
Modelimizi oluşturduktan ve derledikten sonra, sıra onu eğitim verileriyle eğitmeye geliyor. Bu adımda, model belirlenen epochs sayısı kadar eğitim verileri üzerinde çalışacak ve her epoch sonunda test verileriyle doğrulama yaparak performansını değerlendirecek. Model her epoch sırasında ağırlıklarını güncelleyerek daha iyi sonuçlar üretmeye çalışır. Ayrıca, batch_size parametresi, modelin kaç veriyi bir anda işleyeceğini belirler; bu da eğitim sürecinin hızını ve doğruluğunu etkileyebilir.

In [ ]:
# Modelin eğitilmesi
history = model.fit(X_train, y_train, epochs=32, validation_data=(X_test, y_test), batch_size=64)

# 9. Eğitim Performansının Görselleştirilmesi
Eğitim sürecinin ardından, modelin nasıl performans gösterdiğini anlamak için doğruluk ve kayıp değerlerini görselleştiriyoruz. Bu grafikler, modelin eğitim ve doğrulama setlerindeki doğruluğunu ve kayıplarını gösterir. Eğer model, eğitim verisi üzerinde iyi sonuç veriyor fakat doğrulama verisi üzerinde zayıf performans gösteriyorsa, bu durum overfitting'e işaret edebilir. Bu grafikleri inceleyerek, modelin hangi epoch sayısında en iyi sonuçları verdiğini ve overfitting ya da underfitting olup olmadığını gözlemleyebiliriz.

In [ ]:
# Eğitim doğruluğu ve kaybı grafikleri
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Model Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Model Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend()

plt.show()

# 10. Confusion Matrix ve Classification Report
Modelimizin sınıflandırma başarısını daha detaylı incelemek için karışıklık matrisi (Confusion Matrix) ve sınıflandırma raporu (Classification Report) oluşturuyoruz. Karışıklık matrisi, modelin her bir sınıfta ne kadar doğru veya yanlış tahmin yaptığını görselleştirir. Sınıflandırma raporu ise her bir sınıf için precision, recall ve F1-score gibi detaylı metrikleri gösterir. Bu metrikler, modelin başarısını anlamamızda önemli rol oynar. Karışıklık matrisi, doğru sınıflandırılan örneklerin sayısını diyagonal üzerinde gösterirken, yanlış sınıflandırılan örnekler matrisin diğer bölümlerinde gösterilir. Sınıflandırma raporu ise her bir sınıf için detaylı metrikler sunarak modelin performansını ölçmemize yardımcı olur. Bu raporda precision, recall, ve F1-score değerleri yer alır; bu değerler modelin ne kadar iyi bir sınıflandırıcı olduğunu anlamak için kullanılır.

In [ ]:
# Tahminler
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=categories, yticklabels=categories)
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.title('Karışıklık Matrisi')
plt.show()

# Sınıf sayısını kontrol et
unique_classes = np.unique(y_true)
num_classes = len(unique_classes)
print(f"Unique classes in the test set: {num_classes}")

# target_names listesini güncelle
target_names = [categories[i] for i in unique_classes]

# Classification Report
print(classification_report(y_true, y_pred_classes, target_names=target_names, zero_division=0))